# Computer Vision
Een deel van de projecten vragen gebruik van Computer Vision. Deze modellen zijn een stuk kleiner dan LLMs en kan je vaak lokaal draaien en trainen. In dit notebook geef ik een voorbeeld van het lokaal runnen van YOLOv11, een enorm populair Computer Vision Object Detectie model.

In [ ]:
# pip install ultralytics opencv-python # hiermee installeren we YOLOv11 en de benodigde sub-packages en OpenCV

we hebben onderstaande packages nodig

In [ ]:
import cv2
from ultralytics import YOLO
import time

YOLOv11 heeft meerdere modellen:
- nano (n)
- small (s)
- medium (m)
- large (l)
- extra large (x)

Je hoeft deze modellen zelf niet te downloaden, dit doet de code zelf voor je als je hem nog niet hebt.

In [ ]:
model = YOLO("yolo11n.pt")
print("Loaded model:", model)

Oke, nu hebben we het model, laten we met onderstaande functie een scriptje schrijven die onze webcam uitleest en hier alle objecten in detecteert.

In [ ]:
def webcam_yolo_live(model, camera_id=0, confidence_threshold=0.3, wait_time=1/20):
    """
    Yolo inference.

    camera_id: The ID of the camera in your system (0 = default webcam)
    confidence_threshold: display all detections above this threshold
    wait_time: delay to limit GPU usage
    """
    cap = cv2.VideoCapture(camera_id)  
    if not cap.isOpened():
        print("ERROR: Could not open webcam")
        return
    
    try:
        while True:
            ret, frame = cap.read()
            if not ret:
                print("Failed to grab frame")
                break

            # vertaal BGR (OpenCV) -> RGB (YOLO verwacht RGB)
            img = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

            # Stop het plaatje door het model
            results = model(img, verbose=False)

            # Teken de detecties die YOLO heeft gevonden
            annotated = frame.copy()
            for r in results:
                for box in r.boxes:    # r.boxes bevat de detecties
                    # box.xyxy = (xmin, ymin, xmax, ymax) (box afmetingen)
                    x1, y1, x2, y2 = box.xyxy[0].cpu().numpy().astype(int)
                    conf = float(box.conf[0].cpu().numpy())
                    cls  = int(box.cls[0].cpu().numpy())
                    if conf < confidence_threshold:
                        continue

                    label = f"{model.model.names[cls]} {conf:.2f}"
                    cv2.rectangle(annotated, (x1, y1), (x2, y2), (0, 255, 0), 2)
                    cv2.putText(
                        annotated, label, (x1, y1 - 10),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2
                    )

            # Nu kunnen we de output displayen
            cv2.imshow("YOLO Webcam", annotated)

            # Press Q to quit
            if cv2.waitKey(1) & 0xFF == ord('q'):
                break

            time.sleep(wait_time)

    except KeyboardInterrupt:
        print("Stopped the system")

    finally:
        cap.release()
        cv2.destroyAllWindows()

Nu kunnen we de functie aanroepen! Dit opent een nieuw venster. Deze kan je echter niet afsluiten. Om het uit te zetten moet je links naast de cel in het notebook op het vierkantje drukken of bovenin de balk de taak *interrupten*.

In [ ]:
webcam_yolo_live(model)


## Fine-tuning YOLOv11

Het finetunen van YOLO is super makkelijk, maar je moet wel data hebben natuurlijk. Omdat het finetunen gewoon tijd kost en je hier data voor nodig hebt laat ik dat alleen theoretisch zien. De uitvoering zal je zelf moeten zien, maar zo weet je wel wat er gaande is.

### Dataset
Voor YOLO ziet de dataset structuur er zo uit:

```md
dataset/
├── images/
│   ├── train/
│   └── val/
├── labels/
│   ├── train/
│   └── val/
└── config.yaml/
```

In de images folder komen uiteraard de plaatjes. In de labels folder komen .txt files met daarin de label informatie.

In YOLO, die labels zien er zo uit:
```md
class x-center y-center width height
```
example:
```md
0 102 148 10 30
```
Het is hierbij belangrijk dat elk plaatje een file in de labels heeft met exact dezelfde filenaam

###  YAML
Om YOLO precies te vertellen wat er moet gebeuren moet je YOLO wel vertellen hoe je folders er uit zien. Dit doe je met een yaml bestand. Die kan er bijvoorbeeld zo uitzien

Example `config.yaml`:
```yaml
path: small_dataset
train: images/train
val: images/val
names:
  0: human
```

### Model trainen
nu kunnen we het model trainen/fine-tunen, wat ook super simpel is

In [ ]:
results = model.train(
    data="config.yaml",
    epochs=50,
    imgsz=640,
    lr0=0.001,
    batch=1,
)